In [ ]:
from clique_benchmark import run_benchmark_sample, clique_oracle_compiler_classical_function, Trials, CompileType, backend, direct_clique_oracle_circuit
from graph_database import Graphs, Graph
from qiskit_ibm_runtime import Batch

graph_db = Graphs()

In [ ]:
# run benchmark

COMPILE_TYPE = CompileType.CLASSICAL_FUNCTION

trial_db = Trials()

for n in range(3,12):
    # collect the graphs that are already in the database
    # for small optimization (to reduce number of empty batches)
    existing_graphs = set()
    for trial in trial_db.get(n=n, compile_type=COMPILE_TYPE):
        existing_graphs.add(trial.graph.g)

    with Batch(backend) as batch:

        for p in range(1, 100, 2):
            graphs = list(filter(lambda g: g.g not in existing_graphs, graph_db.get(n=n, p=p)))
            if len(graphs) == 0:
                continue

            at_least_one = False
            for graph in graphs:
                target_clique_size = max(n // 2, 2)  # search for big clique for now
                if graph.clique_counts[target_clique_size] > 0:
                    at_least_one = True
                    break

            # avoid empty batch
            if not at_least_one:
                continue

        
            for graph in graphs:
                target_clique_size = max(n // 2, 2)  # search for big clique for now
                if graph.clique_counts[target_clique_size] == 0:
                    continue
                print(f"Running benchmark for graph {graph.g} with clique size {target_clique_size}")
                if COMPILE_TYPE == CompileType.CLASSICAL_FUNCTION:
                    clique_oracle = clique_oracle_compiler_classical_function(graph.g, target_clique_size)
                elif COMPILE_TYPE == CompileType.DIRECT:
                    clique_oracle = direct_clique_oracle_circuit(graph.g, target_clique_size)

                optimal_grover_iterations = graph.optimal_grover_iterations(target_clique_size)

                for grover_iterations in range(1, optimal_grover_iterations + 1):
                    run_benchmark_sample(
                        graph=graph,
                        compile_type=COMPILE_TYPE,
                        clique_oracle=clique_oracle,
                        clique_size=target_clique_size,
                        grover_iterations=grover_iterations,
                        shots=10**3,
                        include_existing_trials=True
                    )
            
        
    
